This notebook is a - kind of - continuation from: inside_bar_explore.ipynb

In [3]:
import pandas as pd
import plotly.graph_objects as go
import utils
import datetime as dt
from dateutil.parser import *

In [4]:
df_trades = pd.read_csv("./Data/Pairs/USD_JPY_H4_trades.csv")

In [5]:
pair = "USD_JPY"
granularity = "M5"

In [7]:
df_raw = pd.read_csv(utils.get_hist_data_filename(pair, granularity))
df_raw.shape

(223553, 16)

In [8]:
non_nums = ['ticker', 'time', 'volume']
num_cols = [x for x in df_raw.columns if x not in non_nums]
df_raw[num_cols] = df_raw[num_cols].apply(pd.to_numeric)

In [10]:
df_trades["time"] = [parse(x) for x in df_trades.time]
df_raw["time"] = [parse(x) for x in df_raw.time]

We need to know the time of the next trade (shift), and the time of the last 5 minute trade in the 4 hour window (because the "H4" granularity data is what we're digging into).  So, that's why we're adding 03:55 to the start time of the 4 hour window.

In [16]:
df_trades["next"] =  df_trades["time"].shift(-1)
df_trades["trade_end"] = df_trades.next + dt.timedelta(hours=3, minutes=55)
df_trades[["time", "next", "trade_end"]].head()

,time,next,trade_end
0,2020-01-05 22:00:00+00:00,2020-01-06 22:00:00+00:00,2020-01-07 01:55:00+00:00
1,2020-01-06 22:00:00+00:00,2020-01-07 06:00:00+00:00,2020-01-07 09:55:00+00:00
2,2020-01-07 06:00:00+00:00,2020-01-09 18:00:00+00:00,2020-01-09 21:55:00+00:00
3,2020-01-09 18:00:00+00:00,2020-01-16 10:00:00+00:00,2020-01-16 13:55:00+00:00
4,2020-01-16 10:00:00+00:00,2020-01-20 14:00:00+00:00,2020-01-20 17:55:00+00:00
